In [281]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
from sklearn.model_selection import train_test_split
import random as rnd

In [282]:
#data = pd.read_excel("C:/Users/Niku/Documents/dataset/arrays.xlsx")
data = pd.read_excel("C:/Users/Niku/Documents/Semester8/DataFiles/datasetnorm1.xlsx")
data.shape

(1994, 121)

In [283]:
def kernel_linear(x1, x2):
    return np.dot(x1, x2.T)
def kernel_quadratic(x1, x2):
    return (np.dot(x1, x2.T) ** 2)
def rnd_int(a,b,z):
    i = z
    while i == z:
        i = rnd.randint(a,b)
    return i
def predictSMO(X):
    return h(X, w, b)
def calc_b(X, y, w):
    b_tmp = y - np.dot(w.T, X.T)
    return np.mean(b_tmp)
def calc_w(alpha, y, X):
    return np.dot(alpha * y, X)
def h(X, w, b):
    return np.sign(np.dot(w.T, X.T) + b).astype(int)
def E(x_k, y_k, w, b):
    return h(x_k, w, b) - y_k
def compute_L_H(C, alpha_prime_j, alpha_prime_i, y_j, y_i):
    if(y_i != y_j):
        return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
    else:
        return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))
def trainSMO(X, y):
    # Initialization
    n, d = X.shape[0], X.shape[1]
    alpha = np.zeros((n))
    #alpha = np.array(alpha)
    #print (alpha.shape)
    count = 0
    while True:
        count += 1
        alpha_prev = np.copy(alpha)
        for j in range(0, n):
            i = rnd_int(0, n-1, j) # Get random int i~=j
            x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
            if kernel == 'quadratic':
                k_ij = kernel_quadratic(x_i, x_i) + kernel_quadratic(x_j, x_j) - 2 * kernel_quadratic(x_i, x_j)
            else:
                k_ij = kernel_linear(x_i, x_i) + kernel_linear(x_j, x_j) - 2 * kernel_linear(x_i, x_j)
            if k_ij == 0:
                continue
            alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
            (L, H) = compute_L_H(C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                # Compute model parameters
            w = calc_w(alpha, y, X)
            b = calc_b(X, y, w)
            
                # Compute E_i, E_j
            E_i = E(x_i, y_i, w, b)
            E_j = E(x_j, y_j, w, b)

                # Set new alpha values
            alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
            alpha[j] = max(alpha[j], L)
            alpha[j] = min(alpha[j], H)

            alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            # Check convergence
        diff = np.linalg.norm(alpha - alpha_prev)
        if diff < epsilon:
            break

        if count >= max_iter:
            print("Iteration number exceeded the max of %d iterations" % (max_iter))
            return
        # Compute final model parameters
    b = calc_b(X, y, w)
    if kernel == 'linear':
        w = calc_w(alpha, y, X)
    # Get support vectors
    alpha_idx = np.where(alpha > 0)[0]
    support_vectors = X[alpha_idx, :]
    return b, w

In [284]:
def initialiseLMC(X_train):
    c = np.zeros(100000)
    V = np.zeros((100000, X_train.shape[1]-1))
    V = np.insert(V, 0, values=np.ones(V.shape[0]), axis=1)
    epochs = 2
    k = 0
    return V, c, k, epochs

def trainLMC(X_train, y_train, V, c, epochs, m, k):
    m = X_train.shape[0]
    n = X_train.shape[1]
    c = np.zeros(100000)
    #V = np.zeros((996, n))
    #V = np.insert(V, 0, values=np.ones(V.shape[0]), axis=1)
    for e in range(epochs):
        print e
        print V.shape
        for i in range(0, m-1):
            y_pred = np.sign(np.dot(X_train[i], V[k].T))
            if y_pred == y_train[i]:
                c[k+1] = c[k] + 1
            else:
                d = np.asscalar(y_train[i])
                p = V[k] + (d*X_train[i]).T
                V[k+1] = V[k] + (d*X_train[i])
                c[k+1] = 1
                k = k + 1
    print 'Done training' 
    print k
    return V, c, k

def predictLMC(X_test, V, c, k):
    y_LMC = np.zeros(X_test.shape[0])
    a = np.zeros(k)
    for j in range(0, X_test.shape[0]):
        for i in range(0, k):
            a[i] = c[i] * np.sign(np.dot(X_test[j], V[i].T))
            s = np.sum(a)
            y_LMC[j] = np.sign(s)
    print 'Test Data Created'
    return y_LMC

In [285]:
max_iter=10000
kernel ='linear'
C=1.0
epsilon=0.001

In [286]:
Normdata = data
Normdata = np.matrix(Normdata)
for i in range(0, Normdata.shape[1]):
    cur = Normdata[:, i]
    mx = max(cur)*1.0
    mn = min(cur)*1.0
    for j in range(0, len(cur)):
        cur[j] = (cur[j] - mn)/(mx - mn)
    Normdata[:, i] = cur
cols = Normdata.shape[1]
X2 = Normdata[:,1:cols-1]
y2 = Normdata[:,cols-1:cols]
X2 = np.matrix(X2)
y2 = np.matrix(y2)
y2[y2 == 0] = -1

In [306]:
rows = data.shape[0]
cols = data.shape[1]
y1 = data.iloc[:, cols-1]
X1 = data.iloc[:, 1:cols-1]
X1 = np.array(X1)
y1 = np.array(y1)

print X1.shape
print X2.shape
y1[y1 == 0] = -1


(1994L, 119L)
(1994L, 119L)


In [307]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 0.5, random_state = 2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 0.5, random_state = 2)

In [320]:
[b, w] = fit(X_train1, y_train1)
y_SMO = predict(X_test1)
u1, c1 = np.unique(y_SMO, return_counts=True)
print u1, c1
u2, c2 = np.unique(np.array(y_test1), return_counts=True)
print u2, c2
correct = [1 if a == b else 0 for (a, b) in zip(y_SMO, y_test1)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)

[-1  1] [579 418]
[-1  1] [594 403]
accuracy = 86.6599799398%


In [321]:
X_train2 = np.insert(X_train2, 0, values=np.ones(X_train2.shape[0]), axis=1)
X_test2 = np.insert(X_test2, 0, values=np.ones(X_test2.shape[0]), axis=1)
X_train1.shape, X_train2.shape, y_train2.shape, X_test2.shape, y_test2.shape

((997L, 119L), (997L, 121L), (997L, 1L), (997L, 121L), (997L, 1L))

In [313]:
V, c, k, epochs = initialiseLMC(X_train2)
V, c, k = trainLMC(X_train2, y_train2, V, c, epochs, m, k)
y_LMC = predictLMC(X_test2, V ,c, k)
correct = [1 if a == b else 0 for (a, b) in zip(y_LMC, y_test2)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)

0
(100000L, 120L)
1
(100000L, 120L)
Done training
447
Test Data Created
accuracy = 86.1584754263%


In [322]:
u1, c1 = np.unique(y_LMC, return_counts=True)
print u1, c1
u2, c2 = np.unique(np.array(y_test2), return_counts=True)
print u2, c2
u3, c3 = np.unique(np.array(y_SMO), return_counts=True)
print u3, c3

[-1.  0.  1.] [579   1 417]
[-1.  1.] [594 403]
[-1  1] [579 418]


In [323]:
correct = [1 if a == b else 0 for (a, b) in zip(y_LMC, y_SMO)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)

accuracy = 96.890672016%


In [347]:
yy = np.zeros(len(y_test2))
for i in range(0, len(y_test2)):
    if y_LMC[i] + y_SMO[i] == 2:
        yy[i] = 1
    elif y_LMC[i] + y_SMO[i] == -2:
        yy[i] = -1
    else:
        yy[i] = y_SMO[i]

In [348]:
correct = [1 if a == b else 0 for (a, b) in zip(yy, y_test2)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)

accuracy = 86.6599799398%
